# Lab 04 - Keypoints

## Lab Exercises

This notebook contains lab exercises for Keypoints.

The keypoints in an image are pixels that are more “important” than  other  pixels,  their 
neighborhood  contains  more  information.  
***
These  points  in  an  image  should  not  depend  on 
- illumination, 
- scale or geometrical, 
- affine distortions (rotations, translations, ...). 

There are many methods that compute keypoints, such that: 
- Harris corner detection, 
- SURF, 
- SIFT, 
- ORB, 
- BRISK, 
- AKAZE,  
- and  so  on.  

Using  keypoint  extraction  was  applied  in  solving  problems  like  
- human action recognition, 
- human pose estimation, 
- object detection and recognition, 
- panorama stitching, 
- video tracking, 
- and this list can continue. 


***

Keypoints are employed in the following way: 
- usually in  the  same  time  with  keypoints  computations,  
    - descriptors  with  information  around  these keypoints  are  also  calculated.  
        - These  descriptors  are  feature  vectors  of  fixed  length  (64  or  128) with information on 
            - orientations, 
            - magnitude of gradient in a neighborhood 
                - around the keypoint.  
- When  comparing  two  images,  
    - a  keypoint  matching  procedure  is  performed  
        - to  identify  the common parts.  

A keypoint approach for solving a given task has the following steps: 
1. Covert to grayscale (if necessary); 
2. Keypoint detection; 
3. Descriptors computation; 
4. (Keypoints, descriptors) matching; 
5. Analyze  the  results  provided  by  the  previous  steps  in  the  context  of  the  problem  to  be 
solved. 

***

Problems to be approached for this lab: 


2. Consider  the  following  dataset  (link  here1).  It  contains  images  from  three  locations  in 
Iaşi. 
a. For  all  the  images  in  this  dataset  compute  the  keypoints  and  the  corresponding 
descriptors, using different methods for keypoints detection. 
b. For each image in the test set perform the matching process with all the images from 
the train set. Use both matching procedures implemented in OpenCV. 
c. Compute  the  set  of  images  from  the  training  set  that  have  maximum  number  of 
matched keypoints with the test image. 
d. If the set computed in step c. has only one image, that image provides the label for the 
test  image.  If  the  set  contains  more  than  one  image,  perform  a  voting  procedure  to 
assign a label (location) to the test image. In case of parity, assign the label randomly 
or find another way to assign the label. Display/print the name of the image for which 
the parity situation occurred. 
e. Compute the accuracy of the classification process and the confusion matrix. 
Remarks: 1) Test different keypoints extracting methods and different matching 
procedures. 
2)  If,  for  an  image,  the  keypoints  detection  method  does  not  compute  any  keypoint, 
modify  some  parameters  of  this  method  until  it  computes  at  least  one  keypoint.  If  for 
certain keypoints methods this type of computation isn’t possible, display a message that 
states  this  fact,  and  continue  with  the  computations.  A  test  image  with  no  keypoints 
cannot  be  classified  (or  assign  a  random  label)  or  it  cannot  be  used  in  the  classification 
process if the image belongs to the training set. 
Useful links: 
https://www.cs.utah.edu/~srikumar/cv_spring2017_files/Keypoints&Descriptors.pdf 
https://www.mdpi.com/2223-7747/10/9/1791/pdf 
OpenCV:  
https://opencv24-python-
tutorials.readthedocs.io/en/latest/py_tutorials/py_feature2d/py_table_of_contents_feature2d/py_t
able_of_contents_feature2d.html 
 1 The images from this archive are part of a larger dataset created by the researchers of the Institute of 
Computer Science, Romanian Academy, Iaşi branch. 
 
https://opencv24-python-
tutorials.readthedocs.io/en/latest/py_tutorials/py_feature2d/py_matcher/py_matcher.html 
https://machinelearningmastery.com/opencv_sift_surf_orb_keypoints/ 
https://learnopencv.com/object-keypoint-similarity/ 
Original images 
 
 
 
 
 
 
Images with SURF keypoints 
 
 
 
 
 
Images with matched keypoints 
 
 
 
 
 
 
5 matched keypoints 
 
 
 
  
 
 
 

1. Compute keypoints for an image using different methods (Harris corner detection, SIFT, 
SURF, ...).  Use as many keypoints detection methods as the OpenCV library allows (at 
least 3). 

In [1]:
# Common setup for Lab 04 - Keypoints
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Create available detectors
def create_detectors():
    dets = {}
    # Harris handled separately
    dets['HARRIS'] = 'HARRIS'
    # SIFT
    try:
        dets['SIFT'] = cv2.SIFT_create()
    except Exception:
        try:
            dets['SIFT'] = cv2.xfeatures2d.SIFT_create()
        except Exception:
            pass
    # SURF (may be unavailable)
    try:
        dets['SURF'] = cv2.xfeatures2d.SURF_create()
    except Exception:
        pass
    # ORB
    try:
        dets['ORB'] = cv2.ORB_create(nfeatures=1000)
    except Exception:
        pass
    # BRISK
    try:
        dets['BRISK'] = cv2.BRISK_create()
    except Exception:
        pass
    # AKAZE
    try:
        dets['AKAZE'] = cv2.AKAZE_create()
    except Exception:
        pass
    return dets

DETECTORS = create_detectors()

# Harris corner detector returning list of cv2.KeyPoint
def harris_keypoints(img_gray, blockSize=2, ksize=3, k=0.04, thresh=0.01):
    dst = cv2.cornerHarris(np.float32(img_gray), blockSize, ksize, k)
    dst = cv2.dilate(dst, None)
    keypoints = []
    # normalize and threshold
    maxv = dst.max() if dst.size else 0
    if maxv == 0:
        return []
    # pick points greater than thresh*max
    ys, xs = np.where(dst > thresh * maxv)
    for (x, y) in zip(xs, ys):
        keypoints.append(cv2.KeyPoint(float(x), float(y), _size=ksize))
    return keypoints

# Generic detect+compute wrapper
def detect_and_compute(detector, img_gray):
    if detector == 'HARRIS':
        kp = harris_keypoints(img_gray)
        # Harris has no descriptors
        return kp, None
    try:
        kp, des = detector.detectAndCompute(img_gray, None)
    except Exception:
        # some detectors may not implement detectAndCompute signature exactly
        try:
            kp = detector.detect(img_gray)
            des = None
        except Exception:
            kp, des = [], None
    return kp, des

# Draw keypoints with matplotlib (RGB display)
def draw_keypoints(img_bgr, keypoints, title=None, figsize=(8,6)):
    img_kp = cv2.drawKeypoints(img_bgr, keypoints, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    img_rgb = cv2.cvtColor(img_kp, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=figsize)
    plt.axis('off')
    if title:
        plt.title(title)
    plt.imshow(img_rgb)
    plt.show()

# Count keypoints in an MxN grid
def keypoints_grid_count(keypoints, img_shape, grid=(8,8)):
    h, w = img_shape[:2]
    gx, gy = grid
    counts = np.zeros((gy, gx), dtype=int)
    for kp in keypoints:
        x, y = int(kp.pt[0]), int(kp.pt[1])
        ix = min(x * gx // w, gx - 1)
        iy = min(y * gy // h, gy - 1)
        counts[iy, ix] += 1
    return counts

# Blurring / rotation / noise helpers
def blur_mean(img, ksize=(5,5)):
    return cv2.blur(img, ksize)

def blur_gaussian(img, ksize=(5,5), sigma=1.0):
    return cv2.GaussianBlur(img, ksize, sigma)

def rotate_image(img, angle_deg):
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((w/2, h/2), angle_deg, 1.0)
    return cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_LINEAR)

def add_gaussian_noise(img, sigma=10):
    if img.dtype != np.uint8:
        img_u8 = (np.clip(img, 0, 1) * 255).astype(np.uint8)
    else:
        img_u8 = img.copy()
    noise = np.random.normal(0, sigma, img_u8.shape).astype(np.int16)
    out = img_u8.astype(np.int16) + noise
    out = np.clip(out, 0, 255).astype(np.uint8)
    return out

# Matching utilities
def _choose_norm(des1, des2):
    # float descriptors (SIFT/SURF) -> L2, binary descriptors -> Hamming
    if des1 is None or des2 is None:
        return cv2.NORM_L2
    if des1.dtype == np.float32 or des2.dtype == np.float32:
        return cv2.NORM_L2
    return cv2.NORM_HAMMING

def bf_match(des1, des2, cross_check=False):
    norm = _choose_norm(des1, des2)
    bf = cv2.BFMatcher(norm, crossCheck=cross_check)
    if des1 is None or des2 is None:
        return []
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)
    return matches

def knn_match(des1, des2, ratio=0.75, k=2):
    norm = _choose_norm(des1, des2)
    bf = cv2.BFMatcher(norm, crossCheck=False)
    if des1 is None or des2 is None:
        return []
    raw = bf.knnMatch(des1, des2, k=k)
    good = []
    for m_n in raw:
        if len(m_n) != 2:
            continue
        m, n = m_n
        if m.distance < ratio * n.distance:
            good.append(m)
    return good

# Compute keypoints and descriptors for all available detectors
def compute_all_detectors(img_bgr):
    img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    results = {}
    for name, det in DETECTORS.items():
        kp, des = detect_and_compute(det, img_gray) if det != 'HARRIS' else (harris_keypoints(img_gray), None)
        if not kp:
            print(f"Warning: detector {name} returned 0 keypoints")
        results[name] = {'keypoints': kp, 'descriptors': des}
    return results

# Example loader for local image path
def load_image(path):
    img = cv2.imread(str(path))
    if img is None:
        raise FileNotFoundError(f"Could not read image: {path}")
    return img

print('Common setup loaded. Available detectors:', list(DETECTORS.keys()))


Common setup loaded. Available detectors: ['HARRIS', 'SIFT', 'ORB', 'BRISK', 'AKAZE']


    
a. Plot the image with marked keypoints. 


In [2]:
# Experiment cell: compute keypoints, grid counts, and perform blur/rotate/noise matching
from pathlib import Path

def display_grid_counts(counts, title=None):
    plt.figure(figsize=(4,4))
    plt.title(title if title else '')
    plt.imshow(counts, cmap='viridis')
    plt.colorbar()
    plt.show()


def match_and_report(orig_res, mod_res, detector_name, img_orig, img_mod, top_draw=20):
    des1 = orig_res[detector_name]['descriptors']
    des2 = mod_res[detector_name]['descriptors']
    print(f"Detector: {detector_name}")
    if des1 is None or des2 is None:
        print("  No descriptors available for matching (skipping).")
        return
    m_bf = bf_match(des1, des2, cross_check=True)
    m_knn = knn_match(des1, des2, ratio=0.75)
    print(f"  BF matches (crossCheck): {len(m_bf)}")
    print(f"  kNN (ratio test) good matches: {len(m_knn)}")
    # draw top matches for visualization
    if len(m_bf) > 0:
        matches_to_draw = m_bf[:top_draw]
        kp1 = orig_res[detector_name]['keypoints']
        kp2 = mod_res[detector_name]['keypoints']
        img_draw = cv2.drawMatches(img_orig, kp1, img_mod, kp2, matches_to_draw, None,
                                   flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
        plt.figure(figsize=(12,6))
        plt.axis('off')
        plt.imshow(cv2.cvtColor(img_draw, cv2.COLOR_BGR2RGB))
        plt.show()


def run_experiments(image_path):
    img = load_image(image_path)
    print('Image loaded:', image_path)

    # compute for original
    orig_res = compute_all_detectors(img)

    # a) Plot image with keypoints for each detector
    for name, r in orig_res.items():
        print(f"\n{name}: {len(r['keypoints'])} keypoints")
        draw_keypoints(img, r['keypoints'], title=f"{name} - {len(r['keypoints'])} keypoints")
        counts = keypoints_grid_count(r['keypoints'], img.shape, grid=(8,8))
        display_grid_counts(counts, title=f"{name} - 8x8 grid counts")

    # Prepare modifications sets
    blurs = [
        ('mean_k5', blur_mean(img, (5,5))),
        ('mean_k9', blur_mean(img, (9,9))),
        ('gauss_k5_s1', blur_gaussian(img, (5,5), sigma=1.0)),
        ('gauss_k9_s2', blur_gaussian(img, (9,9), sigma=2.0)),
    ]
    rotations = [
        ('rot_15', rotate_image(img, 15)),
        ('rot_45', rotate_image(img, 45)),
        ('rot_90', rotate_image(img, 90)),
    ]
    noises = [
        ('noise_s5', add_gaussian_noise(img, sigma=5)),
        ('noise_s15', add_gaussian_noise(img, sigma=15)),
        ('noise_s30', add_gaussian_noise(img, sigma=30)),
    ]

    # c) Blur experiments: compute detectors on blurred and match against original
    for name_mod, img_mod in blurs:
        print(f"\n=== BLUR: {name_mod} ===")
        mod_res = compute_all_detectors(img_mod)
        for det in orig_res.keys():
            match_and_report(orig_res, mod_res, det, img, img_mod)

    # d) Rotation experiments
    for name_mod, img_mod in rotations:
        print(f"\n=== ROTATION: {name_mod} ===")
        mod_res = compute_all_detectors(img_mod)
        for det in orig_res.keys():
            match_and_report(orig_res, mod_res, det, img, img_mod)

    # e) Noise experiments
    for name_mod, img_mod in noises:
        print(f"\n=== NOISE: {name_mod} ===")
        mod_res = compute_all_detectors(img_mod)
        for det in orig_res.keys():
            match_and_report(orig_res, mod_res, det, img, img_mod)


# Example: change this path to a valid image on disk
example_path = Path.cwd() / 'example.jpg'
if example_path.exists():
    run_experiments(str(example_path))
else:
    print('Please set example_path to a valid image file in the notebook workspace and re-run this cell.')


Please set example_path to a valid image file in the notebook workspace and re-run this cell.


    
b.  Compare the keypoints detection methods analyzing the number of computed keypoints and their positioning in the image. 
For the positioning comparisons, divide the  image  in  8  ×  8  non-overlapping  regions,  and  count  the  number  of  keypoints  in each region. 

In [ ]:
# Part b) Compare detectors: total keypoints and 8x8 grid positioning
from pathlib import Path

def compare_detectors_on_image(image_path, grid=(8,8)):
    img = load_image(image_path)
    results = compute_all_detectors(img)

    for name, r in results.items():
        kp = r['keypoints']
        cnt = len(kp)
        print(f"{name}: {cnt} keypoints")
        # show keypoints over image
        draw_keypoints(img, kp, title=f"{name} - {cnt} keypoints", figsize=(6,4))
        # compute and display 8x8 grid counts
        counts = keypoints_grid_count(kp, img.shape, grid=grid)
        display_grid_counts(counts, title=f"{name} - {grid[0]}x{grid[1]} grid counts")

    return results

# Example usage: change path if needed
example_path = Path.cwd() / 'example.jpg'
if example_path.exists():
    compare_detectors_on_image(str(example_path))
else:
    print('Please set example_path to a valid image file in the notebook workspace and re-run this cell.')



c. Blur the image. Compute its keypoints. Match the keypoints of the original image and the blurred one using brute force and the kNN based methods. 
Monitor the number of matched keypoints. 
Use different types of blurring (with mean filters, with Gaussian filters of different size). 

In [ ]:
# Part c) Blur experiments: compute keypoints for blurred images and match to original
from pathlib import Path

def blur_experiments(image_path, top_draw=20):
    img = load_image(image_path)
    print('Image loaded for blur experiments:', image_path)

    # compute for original
    orig_res = compute_all_detectors(img)

    # Different blurring operations
    blurs = [
        ('mean_k5', blur_mean(img, (5,5))),
        ('mean_k9', blur_mean(img, (9,9))),
        ('gauss_k5_s1', blur_gaussian(img, (5,5), sigma=1.0)),
        ('gauss_k9_s2', blur_gaussian(img, (9,9), sigma=2.0)),
    ]

    for name_mod, img_mod in blurs:
        print(f"\n=== BLUR: {name_mod} ===")
        mod_res = compute_all_detectors(img_mod)

        for det in orig_res.keys():
            kp1 = orig_res[det]['keypoints']
            kp2 = mod_res[det]['keypoints']
            des1 = orig_res[det]['descriptors']
            des2 = mod_res[det]['descriptors']
            n1 = len(kp1)
            n2 = len(kp2)

            # If descriptors are missing, skip matching but report counts
            if des1 is None or des2 is None:
                print(f"  {det}: kp_orig={n1}, kp_blur={n2} -> no descriptors, skipping matching")
                continue

            # BF match with cross-check and kNN with ratio test
            m_bf = bf_match(des1, des2, cross_check=True)
            m_knn = knn_match(des1, des2, ratio=0.75)

            print(f"  {det}: kp_orig={n1}, kp_blur={n2}, BFmatches={len(m_bf)}, kNNmatches={len(m_knn)}")

            # Visualize top BF matches if available
            if len(m_bf) > 0:
                matches_to_draw = m_bf[:top_draw]
                img_draw = cv2.drawMatches(img, kp1, img_mod, kp2, matches_to_draw, None,
                                           flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
                plt.figure(figsize=(10,5))
                plt.title(f"{det} matches - {name_mod} (BF top {len(matches_to_draw)})")
                plt.axis('off')
                plt.imshow(cv2.cvtColor(img_draw, cv2.COLOR_BGR2RGB))
                plt.show()


# Example usage: change path if needed
example_path = Path.cwd() / 'example.jpg'
if example_path.exists():
    blur_experiments(str(example_path))
else:
    print('Please set example_path to a valid image file in the notebook workspace and re-run this cell.')


d. Rotate  the  original  image  and  then  perform  the  same  computations/comparisons as those described in step c. 

In [4]:
# Part d) Rotation experiments: rotate image, compute keypoints/descriptors and match against original
from pathlib import Path

def rotation_experiments(image_path, angles=(15,45,90), top_draw=20):
    img = load_image(image_path)
    print('Image loaded:', image_path)

    # compute for original
    orig_res = compute_all_detectors(img)

    for ang in angles:
        print(f"\n=== ROTATION: {ang} degrees ===")
        img_rot = rotate_image(img, ang)
        mod_res = compute_all_detectors(img_rot)

        for det_name in orig_res.keys():
            kp1 = orig_res[det_name]['keypoints']
            kp2 = mod_res[det_name]['keypoints']
            des1 = orig_res[det_name]['descriptors']
            des2 = mod_res[det_name]['descriptors']

            print(f"Detector: {det_name} | orig_kp={len(kp1)} rot_kp={len(kp2)}")
            if des1 is None or des2 is None:
                print("  No descriptors available for matching (skipping).")
                continue

            m_bf = bf_match(des1, des2, cross_check=True)
            m_knn = knn_match(des1, des2, ratio=0.75)
            print(f"  BF matches (crossCheck): {len(m_bf)}")
            print(f"  kNN (ratio test) good matches: {len(m_knn)}")

            # draw top BF matches
            if len(m_bf) > 0:
                matches_to_draw = m_bf[:top_draw]
                img_draw = cv2.drawMatches(img, kp1, img_rot, kp2, matches_to_draw, None,
                                           flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
                plt.figure(figsize=(12,6))
                plt.axis('off')
                plt.title(f"{det_name} - top {len(matches_to_draw)} matches (rot {ang} deg)")
                plt.imshow(cv2.cvtColor(img_draw, cv2.COLOR_BGR2RGB))
                plt.show()

# Example usage: change path if needed
example_path = Path.cwd() / 'example.jpg'
if example_path.exists():
    rotation_experiments(str(example_path), angles=(15,45,90), top_draw=20)
else:
    print('Please set example_path to a valid image file in the notebook workspace and re-run this cell.')


Please set example_path to a valid image file in the notebook workspace and re-run this cell.



e. Add Gaussian noise to the original image. Perform the same analysis as that described in  step  c.  Use  different  amount  of  noise  (different  values  for  the  standard  deviation, Add different noise to an image | TheAILearner). 

In [ ]:
# Part e) Noise experiments: add Gaussian noise to the image and match to original
from pathlib import Path

def noise_experiments(image_path, sigmas=(5, 15, 30), top_draw=20):
    img = load_image(image_path)
    print('Image loaded for noise experiments:', image_path)

    # compute for original
    orig_res = compute_all_detectors(img)

    # prepare noisy versions
    noises = [(f'noise_s{int(s)}', add_gaussian_noise(img, sigma=s)) for s in sigmas]

    for name_mod, img_mod in noises:
        print(f"\n=== NOISE: {name_mod} ===")
        mod_res = compute_all_detectors(img_mod)

        for det in orig_res.keys():
            kp1 = orig_res[det]['keypoints']
            kp2 = mod_res[det]['keypoints']
            des1 = orig_res[det]['descriptors']
            des2 = mod_res[det]['descriptors']
            n1 = len(kp1)
            n2 = len(kp2)

            if des1 is None or des2 is None:
                print(f"  {det}: kp_orig={n1}, kp_noise={n2} -> no descriptors, skipping matching")
                continue

            # BF match with cross-check and kNN with ratio test
            m_bf = bf_match(des1, des2, cross_check=True)
            m_knn = knn_match(des1, des2, ratio=0.75)

            print(f"  {det}: kp_orig={n1}, kp_noise={n2}, BFmatches={len(m_bf)}, kNNmatches={len(m_knn)}")

            # Visualize top BF matches if available
            if len(m_bf) > 0:
                matches_to_draw = m_bf[:top_draw]
                img_draw = cv2.drawMatches(img, kp1, img_mod, kp2, matches_to_draw, None,
                                           flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
                plt.figure(figsize=(10,5))
                plt.title(f"{det} matches - {name_mod} (BF top {len(matches_to_draw)})")
                plt.axis('off')
                plt.imshow(cv2.cvtColor(img_draw, cv2.COLOR_BGR2RGB))
                plt.show()


# Example usage: change path if needed
example_path = Path.cwd() / 'example.jpg'
if example_path.exists():
    noise_experiments(str(example_path))
else:
    print('Please set example_path to a valid image file in the notebook workspace and re-run this cell.')